In [1]:
import numpy as np
import pandas as pd
import argparse
import os

import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F

from custom_dataset import CustomDataset
from network import Network
from utils import *

from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import roc_auc_score,balanced_accuracy_score

In [2]:
########################
# Loading the Data #####
########################



torch.manual_seed(52)
# number of subprocesses to use for data loading
num_workers = 4
# how many samples per batch to load
batch_size = 15
if torch.cuda.device_count() > 1:
    batch_size *= torch.cuda.device_count()
else:
    batch_size = 5
# percentage of training set to use as validation
valid_size = 0.20
# percentage of data to be used for testset
test_size = 0.10


train_data = CustomDataset(transform = 
                        transforms.Compose([
                            transforms.RandomHorizontalFlip()
                            ]),train=True)

valid_data = CustomDataset(train=False,valid=True)

test_data = CustomDataset(train=False,valid=False)

# get filtered variables
vars = valid_data.vars

In [4]:
sss = StratifiedShuffleSplit(n_splits=5,test_size=0.2,random_state=52)
learning_rate = 0.0001
fold = 1
parent_directory = '/data/users2/pnadigapusuresh1/JobOutputs'

split =  sss.split(np.zeros_like(vars),vars.new_score.values)

train_idx , valid_idx = next(iter(split))
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)
test_sampler = SubsetRandomSampler(test_data.test_idx)

train_loader = DataLoader(train_data,batch_size=batch_size, 
                            sampler= train_sampler, num_workers=num_workers)
valid_loader = DataLoader(valid_data,batch_size=batch_size, 
                            sampler= valid_sampler, num_workers=num_workers)

test_loader = DataLoader(test_data,batch_size=batch_size,
                            sampler= test_sampler, num_workers=num_workers)

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

model = Network()
model.fc1 = nn.Sequential(nn.Linear(512,2))

model = nn.DataParallel(model)

load_path = os.path.join(parent_directory,'1818979','models_fold','1','epoch_37')
model.load_state_dict(torch.load(load_path))


Using cuda device


<All keys matched successfully>

In [6]:
for i in range(50):
    try:
        load_path = os.path.join(parent_directory,'1818979','models_fold','1','epoch_'+str(i))
        model.load_state_dict(torch.load(load_path))
    except:
        continue
    

    actual_train = torch.tensor([]).to(device)
    actual_valid = torch.tensor([]).to(device)
    actual_test = torch.tensor([]).to(device)
    pred_proba_train = torch.tensor([]).to(device)
    pred_train = torch.tensor([]).to(device)
    pred_valid = torch.tensor([]).to(device)
    pred_proba_valid = torch.tensor([]).to(device)
    pred_test = torch.tensor([]).to(device)
    pred_proba_test = torch.tensor([]).to(device)
    for X,y,_ in train_loader:
        X,y = X.to(device),y.to(device)
        actual_train = torch.cat((actual_train,y),0)
        pred = torch.squeeze(model(torch.unsqueeze(X,1).float()))
        soft_max = F.softmax(pred,dim=1)
        pred_train = torch.cat((pred_train,torch.max(soft_max, dim=1)   [1]),0)
    for X,y,_ in valid_loader:
        X,y = X.to(device),y.to(device)
        actual_valid = torch.cat((actual_valid,y),0)
        pred = torch.squeeze(model(torch.unsqueeze(X,1).float()))
        soft_max = F.softmax(pred,dim=1)
        pred_valid = torch.cat((pred_valid,torch.max(soft_max, dim=1)   [1]),0)
    for X,y,_ in test_loader:
        X,y = X.to(device),y.to(device)
        actual_test = torch.cat((actual_test,y),0)
        pred = torch.squeeze(model(torch.unsqueeze(X,1).float()))
        soft_max = F.softmax(pred,dim=1)
        pred_test = torch.cat((pred_test,torch.max(soft_max, dim=1)   [1]),0)

    ba_train = balanced_accuracy_score(actual_train.detach().cpu().numpy(),pred_train.detach().cpu().numpy())
    ba_valid = balanced_accuracy_score(actual_valid.detach().cpu().numpy(),pred_valid.detach().cpu().numpy())
    ba_test = balanced_accuracy_score(actual_test.detach().cpu().numpy(),pred_test.detach().cpu().numpy())

    print(f'{i+1}: ba_train = {ba_train}, ba_valid = {ba_valid}, ba_test = {ba_test}')

2: ba_train = 0.6359901784473355, ba_valid = 0.6414921900963775, ba_test = 0.6419563279857398
3: ba_train = 0.6997673768901504, ba_valid = 0.6854346869302004, ba_test = 0.6848781937017231
4: ba_train = 0.6791179695385864, ba_valid = 0.6824391689795478, ba_test = 0.6983511586452763
5: ba_train = 0.6699865648572042, ba_valid = 0.675646483971509, ba_test = 0.696301247771836
6: ba_train = 0.6755505670977986, ba_valid = 0.6713238662391205, ba_test = 0.6860219845513963


KeyboardInterrupt: 

In [ ]:
train_idx , valid_idx = next(iter(split))
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)
test_sampler = SubsetRandomSampler(test_data.test_idx)
for i in range(50):
    try:
        load_path = os.path.join(parent_directory,'1818979','models_fold','2','epoch_'+str(i))
        model.load_state_dict(torch.load(load_path))
    except:
        continue
    

    actual_train = torch.tensor([]).to(device)
    actual_valid = torch.tensor([]).to(device)
    actual_test = torch.tensor([]).to(device)
    pred_proba_train = torch.tensor([]).to(device)
    pred_train = torch.tensor([]).to(device)
    pred_valid = torch.tensor([]).to(device)
    pred_proba_valid = torch.tensor([]).to(device)
    pred_test = torch.tensor([]).to(device)
    pred_proba_test = torch.tensor([]).to(device)
    for X,y,_ in train_loader:
        X,y = X.to(device),y.to(device)
        actual_train = torch.cat((actual_train,y),0)
        pred = torch.squeeze(model(torch.unsqueeze(X,1).float()))
        soft_max = F.softmax(pred,dim=1)
        pred_train = torch.cat((pred_train,torch.max(soft_max, dim=1)   [1]),0)
    for X,y,_ in valid_loader:
        X,y = X.to(device),y.to(device)
        actual_valid = torch.cat((actual_valid,y),0)
        pred = torch.squeeze(model(torch.unsqueeze(X,1).float()))
        soft_max = F.softmax(pred,dim=1)
        pred_valid = torch.cat((pred_valid,torch.max(soft_max, dim=1)   [1]),0)
    for X,y,_ in test_loader:
        X,y = X.to(device),y.to(device)
        actual_test = torch.cat((actual_test,y),0)
        pred = torch.squeeze(model(torch.unsqueeze(X,1).float()))
        soft_max = F.softmax(pred,dim=1)
        pred_test = torch.cat((pred_test,torch.max(soft_max, dim=1)   [1]),0)

    ba_train = balanced_accuracy_score(actual_train.detach().cpu().numpy(),pred_train.detach().cpu().numpy())
    ba_valid = balanced_accuracy_score(actual_valid.detach().cpu().numpy(),pred_valid.detach().cpu().numpy())
    ba_test = balanced_accuracy_score(actual_test.detach().cpu().numpy(),pred_test.detach().cpu().numpy())

    print(f'{i+1}: ba_train = {ba_train}, ba_valid = {ba_valid}, ba_test = {ba_test}')